# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [ ]:
!pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.4 MB/s eta 0:00:00


**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [ ]:
def generate_key(l=2048):
    e = 65537

    while True:
        p = cu.getPrime(l // 2)
        q = cu.getPrime(l // 2)
        v1, v2 = pow(2, (l-1)//2), pow(2, l//2)
        if abs(p - q) > pow(2, (l//2)-100) and v1 < p < v2 and v1 < q < v2 and math.lcm(p - 1, q - 1) >= e * pow(2, l//2):
            break

    n = p * q
    f = (p - 1) * (q - 1)
    assert math.gcd(f, e) == 1

    d = cu.inverse(e, f)
    return d,e,n

In [ ]:
d, e, n = generate_key()

In [ ]:
print(d)
print(e)
print(n)

9175934630196788809494710899390017087735578905930184455254747272887161821703118100951715416051693040704428931607649812969198578560100904879095908500471975397245772330894492289515378729364240240547980908707770466315266253682385091079625697303745994918492239816141522214306062178443631065221861703127156446794504544870941990861845722428990129854907841620144185709123213587106887629017950571328593578102344108107068638594422264543506041247440521327396048287784543500468578465433217380224747182185583471260266601734061872179243277451816411046304505917565524257113625617892987573597606677005784018654015379037580790141809
65537
13922378753049195448623764138846218221950424474648018211881982961133627918436756285180177182520252970520122213519714446278704617379574315948078634888999209418189937983280833939389021062794467116840140408713736932233727009945420005419396891332027625803890029189944597429248886349693481713234364736719091819548587515574133555075299275705901255718092383236469009608965807359751163

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [ ]:
def encrypt(m, e,n):
    return pow(m, e, n)

def decrypt(c, d,n) :
    return pow(c, d, n)

def encrypt_raw(m, e,n):
    k = get_len(n)
    c = encrypt(bytes_to_int(m), e,n)
    return int_to_bytes(c, k)

def decrypt_raw(c, d,n):
    k = get_len(n)
    m = decrypt(bytes_to_int(c), d,n)
    return int_to_bytes(m, k)

## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [ ]:
def sign(M, d, n):
    f = hl.sha3_256()
    f.update(M.encode('utf-8'))
    number = int(f.digest().hex(),16)
    sig = pow(number,d,n)
    return M, sig

def verify(M, sig, e, n):
    f = hl.sha3_256()
    f.update(M.encode('utf-8'))
    number = int(f.digest().hex(),16)
    ver = pow(sig, e, n)
    if ver == number:
        return True
    return False

In [ ]:
message = "Hello World"
message, sig = sign(message, d, n)
print(verify(message, sig, e, n))

True


## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [ ]:
from math import ceil

def int_to_bytes(x, xlen):
    return x.to_bytes(xlen, byteorder='big')

def bytes_to_int(m):
    return int.from_bytes(m, byteorder='big')

def mgf1(seed, mlen, hash):
    t = b''
    hlen = len(hash(b''))
    for c in range(0, ceil(mlen / hlen)):
        tmp = int_to_bytes(c, 4)
        t += hash(seed + tmp)
    return t[:mlen]

## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

In [ ]:
import hashlib

def xor(data, mask):
    masked = b''
    ldata = len(data)
    lmask = len(mask)
    for i in range(max(ldata, lmask)):
        if i < ldata and i < lmask:
            masked += (data[i] ^ mask[i]).to_bytes(1, byteorder='big')
        elif i < ldata:
            masked += data[i].to_bytes(1, byteorder='big')
        else:
            break
    return masked

def hash(m):
    return hashlib.sha256(m).digest()

In [ ]:
type(hash(b'alamakota'))

bytes

In [ ]:
import os

def oaep_encode(m, k):
    
    mlen = len(m)
    lhash = hash(b'')
    hlen = len(lhash)
    
    ps = b'\x00' * (k - mlen - 2 * hlen - 2)
    db = lhash + ps + b'\x01' + m
    
    seed = os.urandom(hlen)
    
    db_mask = mgf1(seed, k - hlen - 1, hash)
    masked_db = xor(db, db_mask)
    
    seed_mask = mgf1(masked_db, hlen, hash)
    masked_seed = xor(seed, seed_mask)
    
    return b'\x00' + masked_seed + masked_db

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

In [ ]:
def oaep_decode(em, k):

    em_len = len(em)
    lhash = hash(b'')
    hlen = len(lhash)
    _, masked_seed, masked_db = em[:1], em[1:1 + hlen], em[1 + hlen:]
    
    seed_mask = mgf1(masked_db, hlen, hash)
    
    seed = xor(masked_seed, seed_mask)
    
    db_mask = mgf1(seed, k - hlen - 1, hash)
    
    db = xor(masked_db, db_mask)
    
    _lhash = db[:hlen]

    assert lhash == _lhash
    
    i = hlen
    
    while i < len(db):
        if db[i] == 0:
            i += 1
            continue
        elif db[i] == 1:
            i += 1
            break
        else:
            raise Exception()

    m = db[i:]
    return m

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [ ]:
def get_len(key):
    return key.bit_length() // 8

In [ ]:
def encrypt_oaep(m):
    hlen = 32  # SHA-1 hash length
    k = get_len(n)
    assert len(m) <= k - hlen - 2
    return encrypt_raw(oaep_encode(m, k), e,n)

In [ ]:
def decrypt_oaep(m):
    k = get_len(n)
    hlen = 32
    assert len(m) == k
    assert k >= 2 * hlen + 2
    return oaep_decode(decrypt_raw(m, d,n), k)

In [ ]:
print(type(hex(12)))

<class 'str'>


In [ ]:
m = b'alamakota'
enc = encrypt_oaep(m)
dec = decrypt_oaep(enc)
print(dec)

b'alamakota'


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


In [ ]:
def emsa_encode(m):
    m_hash = hash(m)
    hlen = len(hash(b''))
    salt = os.urandom(hlen)
    slen = hlen
    embits = get_len(n)*8-1
    emlen = get_len(n)
    print(emlen, hlen,slen)
    assert emlen>hlen+slen+2
    
    m_prim = (0x00).to_bytes(8, 'big') + m_hash + salt

    m_hash_prim = hash(m_prim)

    ps = (0x00).to_bytes((emlen - slen - hlen - 2), 'big')

    db = ps + (0x01).to_bytes(1, 'big') + salt

    db_mask = mgf1(m_hash_prim, emlen-hlen-1, hash)

    masked_db = xor(db, db_mask)

    masked_db =cu.long_to_bytes(masked_db[0] & 0b01111111) + masked_db[1:]
    
    return masked_db + m_hash_prim + (0xbc).to_bytes(1, 'big')

## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


In [ ]:
def emsa_decode(m, em):
    m_hash = hash(m)
    hlen = len(hash(b''))
    salt = os.urandom(hlen)
    slen = hlen
    embits = get_len(n)*8-1
    emlen = get_len(n)

    assert emlen > hlen + slen + 2

    if em[-1]!= 0xbc:
        raise Exception()
        
    masked_db, m_hash_prim = em[:emlen - hlen - 1], em[emlen - hlen - 1:emlen - 1]

    assert masked_db[0] & 0b10000000 == 0
        
    db_mask = mgf1(m_hash_prim, emlen - hlen - 1, hash)
    db = xor(masked_db, db_mask)
    db = cu.long_to_bytes(db[0] & 0b01111111) + db[1:]

    assert cu.bytes_to_long(db[:emlen - hlen - slen - 2]) == 0 and db[emlen - hlen - slen - 2] == 1

    salt = db[-slen:]
    M_p = (0x00).to_bytes(8, 'big') + m_hash + salt
    m_hash_bis = hl.sha256(M_p).digest()
    
    assert m_hash_prim == m_hash_bis
    return True

**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [ ]:
m = b'ala ma podpis'
em = emsa_encode(m)

emsa_decode(m ,em)

128 32 32


True

## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [ ]:
RAND= cu.long_to_bytes(cu.getRandomNBitInteger(2048))
KEY = hash(RAND)
print(KEY)
enc_key = encrypt_oaep(KEY)

dec_key= decrypt_oaep(enc_key)
dec_key

b'\x90\xc8\xe9X\xd0\xc4\x1b\xb6\x8f\x96c\xe1\xb5?CW\xd5\xdd\x9a\xe8\xfe\xe0\x97x\xb0\x91e=\xad\xdas5'


b'\x90\xc8\xe9X\xd0\xc4\x1b\xb6\x8f\x96c\xe1\xb5?CW\xd5\xdd\x9a\xe8\xfe\xe0\x97x\xb0\x91e=\xad\xdas5'